In [1]:
# Dependencies
import requests
import json
import gmaps
import numpy as np
import pandas as pd

# Google developer API key
from config import gkey

In [2]:
# import the data we will use to analyze shops
shop_df = pd.read_csv("Resources/MedicallyEndorsedRetailers.csv", encoding="utf-8")

# Visualize
shop_df.dropna(how='all')
shop_df.head()

,License,UBI,Tradename,Privilege,Status,Med Privilege Code,Termination Code,Street Adress,Suite Rm,City,State,County,Zip Code,Date Created,Day Phone,Email
0,415470,6.029980e+15,2020 SOLUTIONS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,2018 IRON ST STE A,,BELLINGHAM,WA,WHATCOM,982264212,20151012,3603938697,TLOZANO36277@YAHOO.COM
1,413314,6.034150e+15,2020 SOLUTIONS ON THE GUIDE,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,5655 GUIDE MERIDIAN STE A,,BELLINGHAM,WA,WHATCOM,982269722,20151010,3607342020,ACCOUNTING@RETAILBYCHAMPION.COM
2,84154,6.033600e+15,25 TREES,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,3213 S 38TH STE C,,TACOMA,WA,PIERCE,984041094,20180116,2533768562,25treesllc@gmail.com
3,422099,6.035980e+15,365 RECREATIONAL CANNABIS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,36711 U.S. HIGHWAY 12,,DAYTON,WA,COLUMBIA,983288622,20170926,4259194183,seantherider@hotmail.com
4,423784,6.034300e+15,365 RECREATIONAL CANNABIS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,17517 15TH AVE NE #B,,SHORELINE,WA,KING,981553801,20160926,4259194183,seantherider@hotmail.com


In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
# Add columns for lat, lng
shop_df["Lat"] = ""
shop_df["Lng"] = ""
shop_df.columns

Index(['License', 'UBI', 'Tradename', 'Privilege', 'Status',
       'Med Privilege Code', 'Termination Code', 'Street Adress', 'Suite Rm',
       'City', 'State', 'County', 'Zip Code', 'Date Created', 'Day Phone',
       'Email', 'Lat', 'Lng'],
      dtype='object')

In [5]:
# remove unnecesary shop information 
shop_df = shop_df[["Tradename","Street Adress","City","State","County","Zip Code","Lat","Lng"]]

# rename columns for typo and better clarity
shop_df = shop_df.rename(columns={"Tradename":"Store Name", "Street Adress":"Address"})
shop_df.tail()

,Store Name,Address,City,State,County,Zip Code,Lat,Lng
284,WASHINGTON O G,5300 17TH AVE NW STE B,SEATTLE,WA,KING,981073815,,
285,WESTERN BUD,11401 RAINIER AVE S,SEATTLE,WA,KING,981783947,,
286,WESTSIDE420 RECREATIONAL,4503 OCEAN BEACH HWY STE 103,LONGVIEW,WA,COWLITZ,986325055,,
287,WHITE RABBIT RETAIL,15928 HWY 99,LYNNWOOD,WA,SNOHOMISH,980870000,,
288,WORLD OF WEED,3202 E PORTLAND AVE,TACOMA,WA,PIERCE,984044929,,


In [6]:
# create a params dict that will be updated with address in each iteration
params = {"key": gkey}

# Loop through the shop_df and run a lat/long search for each address
for index,row in shop_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    address = row["Address"]
    city = row["City"]
    state = row["State"]
    
    # update address key value
    params['address'] = f"{address},{city},{state}"
    
    # make request
    shop_lat_lng = requests.get(base_url, params=params)
    
    # print the url
    # print(shop_lat_lng.url)
    
    #convert to json
    shop_lat_lng = shop_lat_lng.json()
    
    # add lat and lng to df
    shop_df.loc[index, "Lat"] = shop_lat_lng["results"][0]["geometry"]["location"]["lat"]
    shop_df.loc[index, "Lng"] = shop_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [7]:
# Visualize to confirm lat and lng
shop_df.tail()

,Store Name,Address,City,State,County,Zip Code,Lat,Lng
284,WASHINGTON O G,5300 17TH AVE NW STE B,SEATTLE,WA,KING,981073815,47.6674,-122.379
285,WESTERN BUD,11401 RAINIER AVE S,SEATTLE,WA,KING,981783947,47.5009,-122.222
286,WESTSIDE420 RECREATIONAL,4503 OCEAN BEACH HWY STE 103,LONGVIEW,WA,COWLITZ,986325055,46.1625,-122.999
287,WHITE RABBIT RETAIL,15928 HWY 99,LYNNWOOD,WA,SNOHOMISH,980870000,47.8544,-122.292
288,WORLD OF WEED,3202 E PORTLAND AVE,TACOMA,WA,PIERCE,984044929,47.2345,-122.41


In [8]:
# create a list containing all coordinates
coordinates = []

for index,row in shop_df.iterrows():
    coordinates.append([row["Lat"],row["Lng"]])
    
coordinates

[[48.7570594, -122.4659928],
 [48.84634029999999, -122.4863025],
 [47.222817, -122.4788343],
 [46.3111813, -117.9943499],
 [47.7562484, -122.3137876],
 [47.64163139999999, -117.4886216],
 [46.9955685, -122.9069278],
 [47.04852, -122.7954805],
 [47.0044805, -123.4043625],
 [46.1456599, -122.929237],
 [47.04988849999999, -122.9949418],
 [47.04791549999999, -122.7960181],
 [47.0469316, -122.9255023],
 [47.5549709, -122.2901659],
 [47.7465702, -122.3459965],
 [47.8226234, -122.2064337],
 [46.061236, -118.346435],
 [46.5691867, -122.2669023],
 [48.1754335, -117.7254443],
 [47.7391728, -122.290312],
 [46.2197696, -119.7845671],
 [47.6743475, -122.1281923],
 [47.554262, -120.6026507],
 [48.46268329999999, -122.5814243],
 [47.1243041, -119.2904331],
 [47.7069133, -117.1169724],
 [47.94329200000001, -119.006346],
 [47.541448, -120.560674],
 [47.2438673, -123.0984389],
 [47.6166693, -122.1847115],
 [48.0328516, -122.7641775],
 [47.6568115, -122.6867313],
 [48.3005394, -117.7609423],
 [46.9998379

In [29]:
# Customize the size of the figure
figure_layout = {
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}

# Plot the map
fig = gmaps.figure(layout=figure_layout)

# Create a layer
store_layer = gmaps.symbol_layer(coordinates, 
                                 fill_color='rgba(0,150,0,0.4)',
                                 stroke_color='rgba(0,0,150,0.4)',
                                scale=2,)

# Add the layer to the map
fig.add_layer(store_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px'))

In [53]:
# Import sorted crime data to plot heatmap of crime data
crime_df = pd.read_csv("Resources/df_crime_sorted_Total.csv")
#crime_df = crime_df.drop(crime_df.columns[[0]], axis = 1, inplace=True)

crime_df.head()

,Unnamed: 0,Year,County,JRN_TOTAL,NIB_TOTAL,Total Population,Total Crime,Crime %
0,22,2012,ADAMS,0,1915,19050,1915,10.05
1,23,2013,ADAMS,159,1971,19200,2130,11.09
2,24,2014,ADAMS,162,1960,19400,2122,10.94
3,25,2015,ADAMS,163,1607,19410,1770,9.12
4,26,2016,ADAMS,127,1398,19510,1525,7.82


In [54]:
# drop unamed column
crime_df = crime_df.drop(columns="Unnamed: 0")

In [62]:
crime_2016 = crime_df[crime_df["Year"] == 2016]

# Add Lat and Lng columns
crime_2016["Lat"] = ""
crime_2016["Lng"] = ""

# Reset index
crime_2016 = crime_2016.reset_index(drop=True)

crime_2016

/Users/Kenneth/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Kenneth/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Year,County,JRN_TOTAL,NIB_TOTAL,Total Population,Total Crime,Crime %,Lat,Lng
0,2016,ADAMS,127,1398,19510,1525,7.82,,
1,2016,ASOTIN,72,1439,22150,1511,6.82,,
2,2016,BENTON,1016,10870,190500,11886,6.24,,
3,2016,CHELAN,235,3756,75910,3991,5.26,,
4,2016,CLALLAM,165,4359,73410,4524,6.16,,
5,2016,CLARK,692,20517,461010,21209,4.60,,
6,2016,COLUMBIA,0,211,4050,211,5.21,,
7,2016,COWLITZ,437,7616,104850,8053,7.68,,
8,2016,DOUGLAS,113,1795,40720,1908,4.69,,
9,2016,FERRY,0,25,7700,25,0.32,,


In [81]:
lat2016 = []
lng2016 = []

In [89]:
# create a params dict that will be updated with address in each iteration
params = {"key": gkey}

# Loop through the shop_df and run a lat/long search for each address
for index,row in crime_2016.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    county = row["County"]
    
    # update address key value
    params['address'] = f"{county}+COUNTY,Washington"
    
    # make request
    county_lat_lng = requests.get(base_url, params=params)
    
    # print the url
    # print(county_lat_lng.url)
    
    #convert to json
    county_lat_lng = county_lat_lng.json()
    
    # add lat and lng to df
    crime_2016.loc[index, "Lat"] = county_lat_lng["results"][0]["geometry"]["location"]["lat"]
    crime_2016.loc[index, "Lng"] = county_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [91]:
# Visualize df to see coordinates
crime_2016.tail()

,Year,County,JRN_TOTAL,NIB_TOTAL,Total Population,Total Crime,Crime %,Lat,Lng
35,2016,WAHKIAKUM,2,137,4000,139,3.48,46.3259,-123.495
36,2016,WALLA WALLA,100,4387,60730,4487,7.39,46.237,-118.585
37,2016,WHATCOM,315,14348,212540,14663,6.90,48.8787,-121.972
38,2016,WHITMAN,53,1385,47940,1438,3.00,46.8363,-117.589
39,2016,YAKIMA,889,19713,250900,20602,8.21,46.5436,-120.756


In [114]:
# Create a crime rate Heatmap layer
fig = gmaps.figure()

# Store 'Lat' and 'Lng' into locations
locations = crime_2016[["Lat","Lng"]].astype(float)

#Convert crime rate % to float and Store
crime_rate = crime_2016["Crime %"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=crime_rate,
                                dissipating=False, max_intensity=10,
                                point_radius=0.7)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 10
heat_layer.point_radius = 0.7

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [115]:
# combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))